In [ ]:
import os
import pandas as pd

from datetime import datetime, timedelta
from dotenv import load_dotenv
 
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical.stock import StockHistoricalDataClient
from  alpaca.data.timeframe import TimeFrame, TimeFrameUnit

In [ ]:
load_dotenv()

In [ ]:
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_API_SECRET")

In [ ]:
client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

In [ ]:
timeframe = TimeFrame(5, TimeFrameUnit.Minute)
symbol = 'TSLA'
start = datetime.utcnow() - timedelta(days=30)
end=datetime.utcnow() - timedelta(days=1)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [ ]:
source_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

In [ ]:
display(source_df.tail())

In [ ]:
bars_df = source_df.copy()

In [ ]:
# So we want to add the high, low, and open for the 15 minute and 60 minute window
minutes = 5
windows = [3, 12]

In [ ]:
for window in windows:
    bars_df[f"high - {window*minutes}"] = bars_df["high"].rolling(window=window).max()
    bars_df[f"low - {window*minutes}"] = bars_df["low"].rolling(window=window).min()
    bars_df[f"open - {window*minutes}"] = bars_df["open"].shift(periods=window-1)

In [ ]:
display(bars_df.head())
display(bars_df.tail())